# Work on traffic_signs

In [1]:
import tensorflow as tf
print(tf.__version__)

1.15.0


C:\Users\Lenovo\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Lenovo\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Lenovo\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Lenovo\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dty

In [44]:
!pip install pandas
# !pip install scipy

In [16]:
# Libraries 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
import cv2
from PIL import Image
import os
from sklearn.metrics import classification_report
import time
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [17]:
base_path = r"C:\Users\Lenovo\OneDrive\Compression\archive"
save_path = r".\trafficsignnet.model"
plot_path = r".\plot.png"
test_path = r".\Test"
example_path = r".\examples"
# print(base_path)

C:\Users\Lenovo\OneDrive\Compression\archive


In [18]:
# Reading the input images and putting them into a numpy array
data=[]
labels=[]

height = 32
width = 32
channels = 1
classes = 43
n_inputs = height * width*channels
NUM_EPOCHS = 30
INIT_LR = 1e-3
BS = 64

for i in range(classes) :
    path = base_path+"/train/{0}/".format(i)
    print(path)
    Class=os.listdir(path)
    for a in Class:
        try:
            image=cv2.imread(path+a, cv2.IMREAD_GRAYSCALE)
#             print(image)
#             image_from_array = Image.fromarray(image, 'L')
            image = cv2.resize(image, (height, width))
            data.append(np.array(image))
#             print(image)
            labels.append(i)
        except AttributeError:
            print("Error")
#     print(data)
Cells=np.array(data)
labels=np.array(labels)

#Randomize the order of the input images
s=np.arange(Cells.shape[0])
np.random.seed(43)
np.random.shuffle(s)
Cells=Cells[s]
labels=labels[s]

C:\Users\Lenovo\OneDrive\Compression\archive/train/0/
C:\Users\Lenovo\OneDrive\Compression\archive/train/1/
C:\Users\Lenovo\OneDrive\Compression\archive/train/2/
C:\Users\Lenovo\OneDrive\Compression\archive/train/3/
C:\Users\Lenovo\OneDrive\Compression\archive/train/4/
C:\Users\Lenovo\OneDrive\Compression\archive/train/5/
C:\Users\Lenovo\OneDrive\Compression\archive/train/6/
C:\Users\Lenovo\OneDrive\Compression\archive/train/7/
C:\Users\Lenovo\OneDrive\Compression\archive/train/8/
C:\Users\Lenovo\OneDrive\Compression\archive/train/9/
C:\Users\Lenovo\OneDrive\Compression\archive/train/10/
C:\Users\Lenovo\OneDrive\Compression\archive/train/11/
C:\Users\Lenovo\OneDrive\Compression\archive/train/12/
C:\Users\Lenovo\OneDrive\Compression\archive/train/13/
C:\Users\Lenovo\OneDrive\Compression\archive/train/14/
C:\Users\Lenovo\OneDrive\Compression\archive/train/15/
C:\Users\Lenovo\OneDrive\Compression\archive/train/16/
C:\Users\Lenovo\OneDrive\Compression\archive/train/17/
C:\Users\Lenovo\OneD

In [32]:
#Spliting the images into train and validation sets
(X_train,X_val)=Cells[(int)(0.2*len(labels)):],Cells[:(int)(0.2*len(labels))]
X_train = X_train.astype('float32')/255 
X_val = X_val.astype('float32')/255
(y_train,y_val)=labels[(int)(0.2*len(labels)):],labels[:(int)(0.2*len(labels))]

#Using one hote encoding for the train and validation labels
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, 43)
y_val = to_categorical(y_val, 43)

In [49]:
#accounting for the skewed classes
classTotals = y_train.sum(axis=0)
classWeight = dict()
# loop over all classes and calculate the class weight
for i in range(0, len(classTotals)):
    classWeight[i] = classTotals.max() / classTotals[i]

In [31]:
X_train.shape

(31368, 32, 32, 1)

In [51]:
X_val.shape

(7841, 32, 32)

In [20]:
X_train = X_train[:,:,:,np.newaxis]
X_val = X_val[:,:,:,np.newaxis]
# teX = testX[:,:,:,np.newaxis]
# teX = testX[:,:,:,np.newaxis]

In [21]:
X_train.shape

(31368, 32, 32, 1)

In [54]:
#Definition of the DNN model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))

# #Compilation of the model
# model.compile(
#     loss='categorical_crossentropy', 
#     optimizer='adam', 
#     metrics=['accuracy']
# )

In [55]:
import scipy 
#data augmentation
aug = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode="nearest")

# initialize and compiiling
print("[INFO] compiling model...")
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / (NUM_EPOCHS * 0.5))
# model = TrafficSignNet.build(width=32, height=32, depth=1,
#         classes=numLabels)
model.compile(loss="categorical_crossentropy", optimizer=opt,
        metrics=["accuracy"])

#early callback
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                            patience=5,
                                            restore_best_weights=True)

# train the network
print("[INFO] training network...")
history = model.fit(
    aug.flow(X_train, y_train, batch_size=BS),
    validation_data=(X_val, y_val),
    steps_per_epoch=X_train.shape[0] // BS,
    epochs=NUM_EPOCHS,
    class_weight=classWeight,
    callbacks=[callback],
    verbose=1)

[INFO] compiling model...
[INFO] training network...
Epoch 1/30
490/490 [==============================] - 59s 120ms/step - loss: 7.3680 - acc: 0.1776 - val_loss: 1.2269 - val_acc: 0.6310
Epoch 2/30
490/490 [==============================] - 56s 113ms/step - loss: 3.7345 - acc: 0.5097 - val_loss: 0.5566 - val_acc: 0.8419
Epoch 3/30
490/490 [==============================] - 64s 131ms/step - loss: 2.4104 - acc: 0.6612 - val_loss: 0.2698 - val_acc: 0.9467
Epoch 4/30
490/490 [==============================] - 61s 124ms/step - loss: 1.8179 - acc: 0.7401 - val_loss: 0.1782 - val_acc: 0.9653
Epoch 5/30
490/490 [==============================] - 61s 124ms/step - loss: 1.4373 - acc: 0.7964 - val_loss: 0.1303 - val_acc: 0.9732
Epoch 6/30
490/490 [==============================] - 61s 125ms/step - loss: 1.2225 - acc: 0.8253 - val_loss: 0.0920 - val_acc: 0.9798
Epoch 7/30
490/490 [==============================] - 62s 126ms/step - loss: 1.0361 - acc: 0.8480 - val_loss: 0.0937 - val_acc: 0.9798
Ep

490/490 [==============================] - 60s 122ms/step - loss: 0.6861 - acc: 0.8967 - val_loss: 0.0363 - val_acc: 0.9903
Epoch 12/30
490/490 [==============================] - 59s 120ms/step - loss: 0.6255 - acc: 0.9039 - val_loss: 0.0357 - val_acc: 0.9918
Epoch 13/30
490/490 [==============================] - 63s 128ms/step - loss: 0.5938 - acc: 0.9123 - val_loss: 0.0286 - val_acc: 0.9920
Epoch 14/30
490/490 [==============================] - 59s 121ms/step - loss: 0.5409 - acc: 0.9195 - val_loss: 0.0276 - val_acc: 0.9923
Epoch 15/30
490/490 [==============================] - 59s 120ms/step - loss: 0.5233 - acc: 0.9216 - val_loss: 0.0229 - val_acc: 0.9946
Epoch 16/30
490/490 [==============================] - 62s 127ms/step - loss: 0.4961 - acc: 0.9248 - val_loss: 0.0236 - val_acc: 0.9940
Epoch 17/30
490/490 [==============================] - 59s 120ms/step - loss: 0.4530 - acc: 0.9329 - val_loss: 0.0171 - val_acc: 0.9959
Epoch 18/30
490/490 [==============================] - 60s 1

490/490 [==============================] - 69s 140ms/step - loss: 0.3515 - acc: 0.9454 - val_loss: 0.0146 - val_acc: 0.9959
Epoch 23/30
490/490 [==============================] - 65s 133ms/step - loss: 0.3521 - acc: 0.9467 - val_loss: 0.0142 - val_acc: 0.9966
Epoch 24/30
490/490 [==============================] - 64s 131ms/step - loss: 0.3371 - acc: 0.9487 - val_loss: 0.0109 - val_acc: 0.9973
Epoch 25/30
490/490 [==============================] - 66s 134ms/step - loss: 0.3372 - acc: 0.9505 - val_loss: 0.0144 - val_acc: 0.9958
Epoch 26/30
490/490 [==============================] - 65s 133ms/step - loss: 0.3022 - acc: 0.9526 - val_loss: 0.0134 - val_acc: 0.9962
Epoch 27/30
490/490 [==============================] - 65s 132ms/step - loss: 0.2903 - acc: 0.9529 - val_loss: 0.0114 - val_acc: 0.9971
Epoch 28/30
490/490 [==============================] - 64s 130ms/step - loss: 0.2988 - acc: 0.9534 - val_loss: 0.0135 - val_acc: 0.9955
Epoch 29/30
490/490 [==============================] - 62s 1

In [56]:
# #using ten epochs for the training and saving the accuracy for each epoch
# epochs = 20
# history = model.fit(X_train, y_train, batch_size=32, epochs=epochs,
# validation_data=(X_val, y_val))

In [61]:
#Display of the accuracy and the loss values
#import tkinter
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt

plt.figure(0)
plt.plot(history.history['acc'], label='training accuracy')
plt.plot(history.history['val_acc'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()

plt.figure(1)
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
#plt.show()

In [77]:
#Predicting with the test data
y_test=pd.read_csv(base_path+"\Test.csv")
labels=y_test['Path'].values
y_test=y_test['ClassId'].values
print(labels)


data=[]

for f in labels:
    #print(base_path+"/"+f)
    image=cv2.imread(base_path+"/"+f, cv2.IMREAD_GRAYSCALE)
    print(image.shape)
            #image_from_array = Image.fromarray(image, 'L')
    #print(base_path+f)
    #try:
    image = cv2.resize(image, (height, width))
    data.append(np.array(image))
#     except Exception as e:
#         print(str(e))

testX=np.array(data)
testX = testX[:,:,:,np.newaxis]
testX = testX.astype('float32')/255 
pred = model.predict(testX)

['Test/00000.png' 'Test/00001.png' 'Test/00002.png' ... 'Test/12627.png'
 'Test/12628.png' 'Test/12629.png']
(54, 53)
(45, 42)
(52, 48)
(29, 27)
(57, 60)
(56, 52)
(130, 147)
(33, 32)
(50, 45)
(86, 81)
(37, 38)
(44, 45)
(73, 79)
(37, 36)
(41, 43)
(27, 27)
(38, 37)
(33, 32)
(35, 35)
(40, 34)
(33, 32)
(55, 52)
(120, 116)
(33, 32)
(65, 59)
(34, 35)
(49, 56)
(29, 28)
(36, 34)
(35, 38)
(40, 40)
(34, 29)
(54, 53)
(47, 50)
(59, 58)
(34, 33)
(36, 34)
(74, 80)
(31, 30)
(43, 43)
(70, 71)
(35, 37)
(91, 105)
(40, 41)
(66, 75)
(49, 46)
(33, 33)
(31, 31)
(34, 34)
(40, 41)
(27, 27)
(143, 155)
(87, 97)
(47, 46)
(87, 95)
(34, 34)
(34, 37)
(54, 58)
(35, 33)
(37, 36)
(93, 93)
(40, 40)
(54, 53)
(37, 36)
(42, 42)
(51, 52)
(27, 28)
(37, 37)
(40, 40)
(50, 54)
(71, 74)
(71, 67)
(30, 31)
(27, 27)
(48, 48)
(83, 87)
(51, 45)
(35, 33)
(45, 45)
(40, 41)
(42, 47)
(62, 48)
(70, 68)
(35, 35)
(57, 58)
(53, 53)
(105, 98)
(41, 36)
(80, 84)
(37, 32)
(36, 36)
(43, 43)
(40, 42)
(97, 100)
(70, 71)
(29, 29)
(43, 42)
(30, 31)


(37, 35)
(45, 45)
(187, 209)
(48, 46)
(42, 47)
(98, 109)
(62, 30)
(50, 46)
(117, 113)
(30, 32)
(70, 79)
(26, 30)
(26, 28)
(64, 72)
(55, 57)
(32, 31)
(37, 34)
(36, 36)
(45, 42)
(33, 37)
(39, 35)
(49, 49)
(91, 100)
(55, 52)
(41, 39)
(43, 44)
(30, 29)
(64, 31)
(36, 36)
(31, 30)
(54, 55)
(51, 53)
(32, 32)
(58, 64)
(45, 45)
(29, 28)
(35, 34)
(28, 29)
(28, 28)
(60, 63)
(36, 36)
(33, 30)
(55, 54)
(33, 32)
(57, 59)
(56, 48)
(55, 55)
(57, 61)
(28, 27)
(32, 32)
(49, 30)
(41, 37)
(29, 30)
(53, 53)
(43, 43)
(34, 35)
(42, 42)
(32, 30)
(28, 30)
(33, 34)
(84, 85)
(70, 65)
(33, 32)
(48, 49)
(54, 55)
(94, 91)
(47, 45)
(39, 34)
(30, 28)
(42, 41)
(89, 98)
(46, 46)
(93, 93)
(28, 30)
(58, 63)
(62, 60)
(91, 87)
(36, 34)
(39, 44)
(72, 73)
(42, 43)
(71, 67)
(32, 34)
(33, 35)
(30, 32)
(30, 32)
(29, 26)
(38, 35)
(35, 34)
(117, 133)
(38, 39)
(31, 31)
(56, 56)
(26, 25)
(50, 54)
(28, 29)
(69, 68)
(29, 29)
(51, 43)
(41, 40)
(30, 28)
(95, 94)
(34, 33)
(58, 59)
(38, 35)
(121, 115)
(59, 60)
(95, 96)
(47, 51)
(30, 30)


(26, 26)
(28, 28)
(44, 44)
(48, 50)
(38, 38)
(50, 51)
(42, 43)
(47, 47)
(125, 145)
(28, 29)
(47, 43)
(30, 29)
(32, 31)
(32, 30)
(63, 61)
(57, 57)
(37, 36)
(69, 63)
(28, 29)
(30, 29)
(80, 80)
(40, 42)
(45, 38)
(52, 50)
(43, 47)
(35, 36)
(45, 46)
(40, 40)
(26, 26)
(45, 38)
(35, 38)
(27, 28)
(56, 60)
(39, 34)
(48, 45)
(49, 52)
(40, 41)
(128, 140)
(112, 116)
(72, 68)
(27, 28)
(39, 37)
(36, 34)
(38, 39)
(28, 27)
(47, 47)
(63, 66)
(70, 71)
(30, 33)
(65, 56)
(51, 45)
(30, 34)
(32, 30)
(62, 60)
(26, 29)
(57, 58)
(49, 46)
(35, 35)
(52, 53)
(49, 49)
(34, 37)
(45, 44)
(56, 58)
(35, 35)
(53, 53)
(34, 35)
(43, 42)
(38, 40)
(39, 39)
(80, 80)
(74, 65)
(48, 53)
(27, 27)
(38, 37)
(38, 38)
(42, 43)
(44, 41)
(82, 94)
(48, 47)
(86, 82)
(50, 49)
(30, 27)
(39, 45)
(35, 36)
(93, 88)
(37, 38)
(45, 43)
(51, 51)
(38, 40)
(28, 27)
(28, 27)
(31, 31)
(71, 69)
(39, 39)
(44, 44)
(45, 47)
(55, 55)
(46, 51)
(32, 31)
(60, 70)
(52, 49)
(50, 51)
(56, 60)
(48, 48)
(34, 39)
(30, 30)
(38, 39)
(37, 37)
(46, 46)
(36, 37)
(60,

(40, 41)
(43, 44)
(59, 64)
(28, 27)
(32, 36)
(36, 36)
(51, 47)
(42, 42)
(65, 62)
(53, 48)
(39, 39)
(47, 46)
(31, 29)
(27, 31)
(34, 33)
(34, 34)
(50, 52)
(68, 66)
(47, 50)
(27, 27)
(78, 80)
(57, 57)
(74, 74)
(45, 41)
(40, 42)
(81, 76)
(202, 230)
(125, 142)
(40, 39)
(132, 139)
(39, 40)
(31, 31)
(26, 26)
(82, 79)
(64, 61)
(34, 35)
(35, 35)
(41, 41)
(61, 69)
(45, 43)
(96, 88)
(39, 40)
(27, 25)
(60, 67)
(65, 64)
(59, 59)
(41, 39)
(36, 33)
(43, 41)
(32, 34)
(39, 39)
(34, 35)
(136, 141)
(51, 55)
(53, 58)
(60, 64)
(48, 47)
(42, 41)
(55, 55)
(108, 116)
(57, 56)
(57, 56)
(40, 38)
(31, 31)
(61, 64)
(75, 80)
(53, 51)
(34, 34)
(50, 50)
(64, 58)
(58, 58)
(28, 28)
(36, 36)
(62, 62)
(31, 30)
(32, 30)
(28, 26)
(54, 54)
(33, 33)
(59, 58)
(42, 41)
(27, 27)
(44, 39)
(33, 33)
(40, 42)
(37, 39)
(37, 35)
(110, 104)
(104, 117)
(62, 65)
(32, 33)
(33, 31)
(47, 43)
(43, 46)
(37, 38)
(32, 30)
(35, 35)
(80, 72)
(32, 32)
(64, 64)
(32, 33)
(34, 31)
(87, 81)
(32, 32)
(82, 83)
(70, 69)
(47, 29)
(40, 37)
(42, 42)
(40, 

(47, 44)
(31, 31)
(51, 48)
(39, 39)
(36, 36)
(36, 37)
(28, 26)
(30, 29)
(34, 31)
(45, 46)
(64, 64)
(43, 44)
(29, 27)
(47, 49)
(35, 35)
(46, 40)
(34, 33)
(35, 35)
(34, 34)
(50, 51)
(29, 29)
(29, 28)
(34, 34)
(50, 49)
(59, 68)
(155, 169)
(50, 46)
(89, 89)
(57, 58)
(96, 98)
(68, 67)
(48, 47)
(82, 34)
(77, 84)
(43, 48)
(38, 39)
(47, 46)
(31, 30)
(37, 36)
(33, 30)
(48, 38)
(70, 61)
(85, 85)
(44, 47)
(51, 54)
(58, 57)
(63, 60)
(82, 45)
(82, 79)
(40, 45)
(118, 131)
(48, 48)
(111, 107)
(42, 43)
(39, 41)
(45, 42)
(46, 48)
(28, 28)
(51, 54)
(38, 35)
(45, 46)
(69, 69)
(77, 84)
(29, 30)
(99, 99)
(43, 42)
(45, 44)
(41, 41)
(54, 51)
(38, 38)
(35, 37)
(67, 68)
(29, 26)
(36, 34)
(37, 38)
(81, 74)
(63, 64)
(37, 35)
(71, 37)
(29, 27)
(34, 32)
(75, 84)
(115, 128)
(37, 39)
(42, 39)
(49, 49)
(58, 59)
(52, 54)
(75, 75)
(30, 29)
(118, 118)
(27, 27)
(50, 46)
(42, 45)
(35, 34)
(58, 54)
(64, 63)
(31, 31)
(63, 73)
(39, 42)
(35, 36)
(61, 71)
(58, 32)
(48, 44)
(33, 28)
(129, 143)
(37, 36)
(33, 34)
(35, 36)
(132, 1

...

(58, 58)
(50, 58)
(39, 42)
(27, 28)
(41, 27)
(58, 59)
(44, 46)
(39, 38)
(42, 42)
(27, 28)
(41, 43)
(84, 81)
(46, 44)
(85, 94)
(42, 38)
(51, 53)
(48, 44)
(43, 44)
(69, 69)
(80, 85)
(29, 30)
(37, 37)
(146, 147)
(46, 46)
(48, 37)
(35, 39)
(58, 58)
(33, 34)
(59, 62)
(49, 57)
(50, 48)
(52, 54)
(33, 29)
(42, 40)
(51, 54)
(72, 63)
(72, 72)
(35, 36)
(41, 40)
(36, 37)
(42, 43)
(50, 50)
(29, 30)
(63, 72)
(32, 32)
(35, 35)
(36, 38)
(27, 28)
(39, 40)
(35, 34)
(26, 28)
(43, 44)
(29, 30)
(45, 45)
(40, 40)
(41, 41)
(52, 52)
(53, 41)
(75, 83)
(29, 28)
(36, 38)
(54, 51)
(59, 59)
(38, 41)
(63, 57)
(57, 59)
(46, 45)
(55, 50)
(69, 68)
(42, 43)
(46, 47)
(37, 34)
(58, 54)
(69, 68)
(31, 29)
(39, 43)
(48, 45)
(30, 30)
(37, 37)
(28, 27)
(45, 44)
(49, 49)
(26, 25)
(71, 80)
(38, 37)
(55, 56)
(39, 38)
(32, 34)
(61, 64)
(36, 35)
(37, 36)
(36, 36)
(50, 58)
(96, 85)
(38, 38)
(51, 51)
(40, 39)
(49, 49)
(32, 33)
(36, 35)
(99, 105)
(67, 62)
(31, 31)
(52, 45)
(38, 39)
(52, 53)
(36, 39)
(31, 28)
(63, 58)
(80, 87)
(29, 28

(53, 59)
(39, 39)
(31, 31)
(47, 46)
(99, 99)
(32, 31)
(41, 40)
(74, 71)
(79, 80)
(74, 35)
(33, 34)
(34, 37)
(36, 36)
(31, 30)
(33, 32)
(26, 27)
(73, 73)
(41, 41)
(29, 28)
(71, 77)
(35, 35)
(47, 50)
(134, 147)
(31, 31)
(30, 32)
(31, 32)
(49, 50)
(27, 29)
(33, 33)
(138, 137)
(53, 61)
(37, 40)
(120, 123)
(34, 32)
(33, 33)
(40, 36)
(36, 36)
(38, 34)
(40, 39)
(40, 40)
(34, 32)
(39, 40)
(104, 110)
(31, 30)
(28, 27)
(62, 62)
(57, 56)
(50, 46)
(37, 36)
(59, 63)
(34, 33)
(61, 64)
(29, 28)
(47, 49)
(70, 76)
(44, 45)
(94, 87)
(36, 36)
(29, 29)
(34, 39)
(31, 29)
(38, 38)
(43, 44)
(37, 38)
(28, 27)
(30, 31)
(49, 48)
(27, 26)
(62, 62)
(28, 29)
(41, 38)
(40, 44)
(45, 39)
(32, 33)
(28, 28)
(54, 54)
(81, 75)
(54, 56)
(57, 58)
(59, 58)
(35, 36)
(41, 42)
(71, 73)
(52, 50)
(43, 44)
(77, 80)
(76, 72)
(39, 35)
(37, 38)
(34, 38)
(29, 29)
(33, 30)
(35, 34)
(41, 41)
(59, 54)
(121, 120)
(121, 122)
(128, 130)
(46, 46)
(77, 80)
(37, 37)
(36, 35)
(73, 74)
(33, 37)
(60, 60)
(29, 29)
(31, 31)
(35, 31)
(50, 50)
(109,

(32, 35)
(48, 50)
(64, 63)
(50, 53)
(27, 26)
(61, 55)
(37, 37)
(73, 73)
(36, 37)
(31, 31)
(81, 81)
(25, 25)
(63, 60)
(32, 31)
(73, 74)
(39, 38)
(30, 29)
(46, 45)
(30, 30)
(64, 71)
(112, 115)
(75, 75)
(36, 35)
(70, 75)
(119, 116)
(54, 55)
(51, 49)
(31, 29)
(49, 49)
(60, 60)
(57, 57)
(43, 48)
(31, 31)
(30, 32)
(45, 47)
(96, 85)
(86, 84)
(59, 57)
(37, 39)
(95, 88)
(41, 44)
(42, 44)
(33, 32)
(40, 41)
(43, 44)
(65, 72)
(33, 34)
(37, 37)
(105, 54)
(27, 29)
(147, 154)
(31, 29)
(55, 64)
(52, 51)
(31, 32)
(66, 63)
(42, 42)
(65, 69)
(51, 54)
(49, 54)
(26, 28)
(34, 35)
(51, 56)
(42, 44)
(46, 44)
(53, 44)
(74, 68)
(46, 48)
(32, 32)
(40, 38)
(44, 46)
(67, 67)
(31, 28)
(37, 40)
(53, 53)
(55, 56)
(34, 35)
(58, 68)
(34, 35)
(32, 30)
(57, 52)
(69, 71)
(48, 49)
(33, 33)
(28, 27)
(37, 37)
(82, 76)
(39, 39)
(27, 27)
(55, 58)
(32, 33)
(65, 72)
(32, 32)
(34, 35)
(58, 59)
(55, 53)
(54, 55)
(49, 49)
(29, 26)
(32, 32)
(148, 146)
(135, 136)
(113, 125)
(34, 29)
(64, 71)
(35, 39)
(80, 80)
(38, 38)
(45, 42)
(39, 3

(44, 49)
(39, 39)
(54, 53)
(49, 61)
(58, 68)
(55, 60)
(48, 48)
(52, 49)
(35, 36)
(31, 31)
(35, 35)
(84, 82)
(33, 34)
(40, 40)
(59, 57)
(37, 37)
(43, 51)
(123, 113)
(30, 31)
(49, 50)
(117, 118)
(46, 30)
(33, 32)
(32, 33)
(28, 28)
(44, 42)
(39, 38)
(38, 40)
(62, 59)
(105, 104)
(43, 47)
(33, 28)
(35, 31)
(43, 28)
(62, 64)
(63, 61)
(50, 51)
(51, 51)
(51, 50)
(38, 41)
(121, 64)
(30, 35)
(58, 58)
(38, 38)
(47, 47)
(28, 28)
(76, 86)
(44, 42)
(34, 34)
(96, 92)
(31, 32)
(34, 34)
(35, 35)
(77, 71)
(98, 111)
(47, 46)
(34, 36)
(62, 62)
(48, 47)
(40, 41)
(55, 53)
(49, 49)
(31, 29)
(64, 73)
(55, 43)
(55, 51)
(45, 49)
(92, 95)
(34, 35)
(47, 39)
(79, 80)
(34, 32)
(29, 29)
(39, 40)
(76, 77)
(48, 48)
(57, 68)
(69, 72)
(75, 86)
(49, 49)
(37, 33)
(44, 43)
(48, 46)
(38, 41)
(51, 51)
(42, 39)
(68, 73)
(35, 33)
(37, 36)
(57, 55)
(52, 53)
(61, 57)
(33, 36)
(57, 56)
(60, 71)
(42, 36)
(32, 34)
(92, 95)
(35, 35)
(27, 29)
(26, 26)
(45, 49)
(53, 52)
(27, 28)
(42, 42)
(38, 43)
(35, 35)
(43, 41)
(45, 48)
(47, 46)
(3

(48, 47)
(33, 33)
(32, 33)
(31, 32)
(46, 55)
(82, 82)
(41, 39)
(52, 49)
(34, 37)
(80, 76)
(70, 70)
(39, 39)
(34, 34)
(88, 82)
(43, 42)
(32, 33)
(39, 39)
(172, 179)
(29, 26)
(27, 28)
(35, 39)
(45, 46)
(36, 35)
(40, 41)
(55, 56)
(43, 42)
(33, 33)
(92, 93)
(54, 53)
(31, 30)
(53, 49)
(38, 39)
(30, 31)
(116, 132)
(84, 84)
(40, 40)
(61, 62)
(70, 74)
(76, 83)
(61, 64)
(41, 39)
(61, 71)
(37, 37)
(39, 38)
(57, 62)
(37, 35)
(35, 35)
(46, 55)
(42, 47)
(47, 48)
(56, 59)
(36, 38)
(66, 34)
(35, 37)
(53, 53)
(37, 39)
(77, 77)
(27, 26)
(46, 47)
(89, 85)
(44, 44)
(44, 46)
(31, 30)
(44, 43)
(42, 37)
(31, 34)
(30, 31)
(42, 42)
(33, 33)
(40, 40)
(56, 57)
(29, 29)
(48, 47)
(60, 62)
(50, 49)
(41, 43)
(52, 49)
(43, 43)
(28, 26)
(38, 38)
(30, 30)
(79, 79)
(58, 61)
(52, 51)
(51, 56)
(55, 54)
(37, 37)
(29, 29)
(69, 74)
(60, 59)
(36, 38)
(30, 29)
(63, 67)
(76, 41)
(37, 38)
(34, 32)
(34, 37)
(30, 30)
(61, 58)
(50, 47)
(94, 82)
(28, 31)
(46, 50)
(33, 33)
(32, 31)
(39, 38)
(57, 57)
(81, 77)
(41, 40)
(31, 30)
(43, 4

In [78]:
pred = np.argmax(pred,axis=1)

In [79]:
#Accuracy with the test data
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

0.970387965162312

In [80]:
labelNames = open("signnames.csv").read().strip().split("\n")[1:]
labelNames = [l.split(",")[1] for l in labelNames]

In [84]:
timestr = time.strftime("%Y%m%d-%H%M%S")
osdd.path.join(save_path,timestr)

'.\\trafficsignnet.model\\20220125-191042'

In [93]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX)
print(classification_report(y_test,pred, target_names=labelNames))

# save the network
timestr = time.strftime("%Y%m%d-%H%M%S")
print("[INFO] serializing network to '{}/{}'...".format(save_path,"my_model.h5"))
# save_path = os.path.sep.join(save_path,)
model.save(os.path.join(save_path,"my_model.h5"))
# tf.keras.models.save_model(model,os.path.sep.join([save_path,timestr]))

[INFO] evaluating network...
                                                    precision    recall  f1-score   support

                              Speed limit (20km/h)       1.00      0.90      0.95        60
                              Speed limit (30km/h)       0.99      1.00      1.00       720
                              Speed limit (50km/h)       0.97      0.98      0.98       750
                              Speed limit (60km/h)       0.98      0.96      0.97       450
                              Speed limit (70km/h)       1.00      0.95      0.97       660
                              Speed limit (80km/h)       0.96      0.96      0.96       630
                       End of speed limit (80km/h)       0.99      1.00      1.00       150
                             Speed limit (100km/h)       0.98      0.95      0.96       450
                             Speed limit (120km/h)       0.92      1.00      0.96       450
                                        No passing

# Pruning, Quantization and compression

In [2]:
import tensorflow_model_optimization as tfmot
import numpy as np
import tensorflow as tf
import tempfile

In [4]:
loaded_model = tf.keras.models.load_model(save_path+"\my_model.h5")

In [5]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

In [6]:
# Compute end step to finish pruning after 2 epochs.
batch_size = 32
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

In [7]:
num_images = 10360 * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

In [8]:
# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

In [9]:
model_for_pruning = prune_low_magnitude(loaded_model, **pruning_params)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


C:\Users\Lenovo\anaconda3\lib\site-packages\tensorflow_model_optimization\python\core\sparsity\keras\pruning_wrapper.py:218: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
C:\Users\Lenovo\anaconda3\lib\site-packages\tensorflow_model_optimization\python\core\sparsity\keras\pruning_wrapper.py:225: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
C:\Users\Lenovo\anaconda3\lib\site-packages\tensorflow_model_optimization\python\core\sparsity\keras\pruning_wrapper.py:238: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  trainable=False)


In [10]:
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [11]:
model_for_pruning.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_conv2d_  (None, 28, 28, 32)       1634      
 20 (PruneLowMagnitude)                                          
                                                                 
 prune_low_magnitude_conv2d_  (None, 26, 26, 64)       36930     
 21 (PruneLowMagnitude)                                          
                                                                 
 prune_low_magnitude_max_poo  (None, 13, 13, 64)       1         
 ling2d_12 (PruneLowMagnitud                                     
 e)                                                              
                                                                 
 prune_low_magnitude_dropout  (None, 13, 13, 64)       1         
 _8 (PruneLowMagnitude)                                          
                                                      

In [27]:
logdir = tempfile.mkdtemp()

# callbacks = [
#   tfmot.sparsity.keras.UpdatePruningStep(),
#   tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
# ]

model_for_pruning.fit(X_train, y_train,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=[tfmot.sparsity.keras.UpdatePruningStep()])

Epoch 1/2


InvalidArgumentError:  logits and labels must have the same first dimension, got logits shape [32,43] and labels shape [1376]
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits
 (defined at C:\Users\Lenovo\anaconda3\lib\site-packages\keras\backend.py:5114)
]] [Op:__inference_train_function_4122]

Errors may have originated from an input operation.
Input Source operations connected to node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits:
In[0] sparse_categorical_crossentropy/Reshape_1 (defined at C:\Users\Lenovo\anaconda3\lib\site-packages\keras\backend.py:5109)	
In[1] sparse_categorical_crossentropy/Reshape (defined at C:\Users\Lenovo\anaconda3\lib\site-packages\keras\backend.py:3561)

Operation defined at: (most recent call last)
>>>   File "C:\Users\Lenovo\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
>>>     "__main__", mod_spec)
>>> 
>>>   File "C:\Users